<a href="https://colab.research.google.com/github/mohamedabdelaty74/Knooz/blob/main/Promotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
from itertools import combinations, product

def process_cases_from_any_sheet(file_path, sheet_name):
    # Load the Excel file
    data_alotour_new_filled = pd.read_excel("/content/عروض شهر سبتمبر 2024 مالية.xlsx", sheet_name="العطور والبخور")

    # Fill down missing values for all relevant columns
    data_alotour_new_filled.fillna(method='ffill', inplace=True)

    # Define a function to handle pairwise combinations (for cases like case1 and case1.1)
    def process_case_combinations(case_data):
        product_combinations = list(combinations(case_data.iterrows(), 2))
        combined_codes = []
        combined_names = []
        combined_prices_before = []
        combined_prices_after = []
        combined_descriptions = []

        for (i1, prod1), (i2, prod2) in product_combinations:
            code_comb = f"{prod1['مرجع داخلي']}, {prod2['مرجع داخلي']}"  # Internal Reference
            name_comb = f"{prod1['الاسم']} + {prod2['الاسم']}"  # Product Name
            price_before_comb = prod1['السعر قبل الخصم ش.ض'] + prod2['السعر قبل الخصم ش.ض']  # Price Before Discount
            price_after_comb = prod1['السعر بعد الخصم ش.ض'] + prod2['السعر بعد الخصم ش.ض']  # Price After Discount
            description_comb = f"{prod1['الوصف']}, {prod2['الوصف']}"  # Description

            combined_codes.append(code_comb)
            combined_names.append(name_comb)
            combined_prices_before.append(price_before_comb)
            combined_prices_after.append(price_after_comb)
            combined_descriptions.append(description_comb)

        case_combination_df = pd.DataFrame({
            'Combined Codes': combined_codes,
            'Combined Names': combined_names,
            'Combined Price Before Discount': combined_prices_before,
            'Combined Price After Discount': combined_prices_after,
            'Description': combined_descriptions
        })

        case_combination_df['Discount Percentage'] = (
            (case_combination_df['Combined Price Before Discount'] - case_combination_df['Combined Price After Discount'])
            / case_combination_df['Combined Price Before Discount']
        ) * 100

        return case_combination_df

    # Define a function to handle Cartesian product (for cases like case2 and case2.1)
    def process_case_matrix_product(case_a_data, case_b_data):
        product_combinations = list(product(case_a_data.iterrows(), case_b_data.iterrows()))
        combined_codes = []
        combined_names = []
        combined_prices_before = []
        combined_prices_after = []
        combined_descriptions = []

        for (i1, prod1), (i2, prod2) in product_combinations:
            code_comb = f"{prod1['مرجع داخلي']}, {prod2['مرجع داخلي']}"  # Internal Reference
            name_comb = f"{prod1['الاسم']} + {prod2['الاسم']}"  # Product Name
            price_before_comb = prod1['السعر قبل الخصم ش.ض'] + prod2['السعر قبل الخصم ش.ض']  # Price Before Discount
            price_after_comb = prod1['السعر بعد الخصم ش.ض'] + prod2['السعر بعد الخصم ش.ض']  # Price After Discount
            description_comb = f"{prod1['الوصف']}, {prod2['الوصف']}"  # Description

            combined_codes.append(code_comb)
            combined_names.append(name_comb)
            combined_prices_before.append(price_before_comb)
            combined_prices_after.append(price_after_comb)
            combined_descriptions.append(description_comb)

        case_combination_df = pd.DataFrame({
            'Combined Codes': combined_codes,
            'Combined Names': combined_names,
            'Combined Price Before Discount': combined_prices_before,
            'Combined Price After Discount': combined_prices_after,
            'Description': combined_descriptions
        })

        case_combination_df['Discount Percentage'] = (
            (case_combination_df['Combined Price Before Discount'] - case_combination_df['Combined Price After Discount'])
            / case_combination_df['Combined Price Before Discount']
        ) * 100

        return case_combination_df

    # Process Case1 and Case1.1 individually
    individual_cases = ['case1', 'case1.1']
    combined_results = []

    for case in individual_cases:
        case_data = data_alotour_new_filled[data_alotour_new_filled['Status'] == case]

        if not case_data.empty:
            case_combination_df = process_case_combinations(case_data)
            combined_results.append(case_combination_df)

    # Process Case2 and its subcases as Cartesian products
    case_combinations = [('case2', 'case2.1'), ('case2.2', 'case2.3'), ('case2.4', 'case2.5'), ('case2.6', 'case2.7')]

    for case_a, case_b in case_combinations:
        case_a_data = data_alotour_new_filled[data_alotour_new_filled['Status'] == case_a]
        case_b_data = data_alotour_new_filled[data_alotour_new_filled['Status'] == case_b]

        if not case_a_data.empty and not case_b_data.empty:
            case_combination_df = process_case_matrix_product(case_a_data, case_b_data)
            combined_results.append(case_combination_df)

    # Combine all results
    if combined_results:
        final_combined_df = pd.concat(combined_results, ignore_index=True)

        # Adjust "Combined Codes" by splitting them into two rows and adding blanks
        final_combined_df_expanded = pd.DataFrame()

        for index, row in final_combined_df.iterrows():
            codes = row['Combined Codes'].split(',')
            first_row = row.copy()
            second_row = pd.Series([None] * len(row), index=row.index)

            first_row['Combined Codes'] = codes[0].strip()  # Keep the first code
            second_row['Combined Codes'] = codes[1].strip()  # Move the second code to the new row

            final_combined_df_expanded = pd.concat([final_combined_df_expanded, first_row.to_frame().T, second_row.to_frame().T], ignore_index=True)

        # Save the final result to an Excel file
        final_output_path = "Final_Cases_Combined_Output_from_Sheet.xlsx"
        final_combined_df_expanded.to_excel(final_output_path, index=False)

        return final_output_path
    else:
        return "No cases found in the sheet."

# Example usage:
# Provide your Excel file path and the sheet name where data is stored
file_path = "your_file_path_here.xlsx"
sheet_name = "العطور والبخور"  # Replace with the actual sheet name you want to process
final_output_path = process_cases_from_any_sheet(file_path, sheet_name)
print(f"Output saved to: {final_output_path}")


Output saved to: Final_Cases_Combined_Output_from_Sheet.xlsx


<ipython-input-31-c39d2e1b0a6d>:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_alotour_new_filled.fillna(method='ffill', inplace=True)
